
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>




# Feature Store Lab

Now that you are familiar with the <a href="https://docs.databricks.com/applications/machine-learning/feature-store.html" target="_blank">Databricks Feature Store</a>, try applying the concepts we learned to a new dataset below.

The Feature Store Python API documentation can be found <a href="https://docs.databricks.com/dev-tools/api/python/latest/index.html#feature-store-python-api-reference" target="_blank">here</a>.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Learning Objectives:<br>


By the end of this lab, you should be able to;

* Create a feature store and save features to this store
* Update an existing feature store table
* Register a MLflow model with feature store tables
* Perform batch scoring with a feature store

## Lab Setup

The first thing we're going to do is to **run setup script**. This script will define the required configuration variables that are scoped to each user.

In [0]:
%run "../Includes/Classroom-Setup"

Python interpreter will be restarted.
Python interpreter will be restarted.


Resetting the learning environment:
| No action taken

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/scalable-machine-learning-with-apache-spark/v02"

Validating the locally installed datasets:
| listing local files...(3 seconds)
| validation completed...(3 seconds total)

Creating & using the schema "charlie_ohara_4mi2_da_sml" in the catalog "hive_metastore"...(0 seconds)

Predefined tables in "charlie_ohara_4mi2_da_sml":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/charlie.ohara@standard.ai/scalable-machine-learning-with-apache-spark
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/charlie.ohara@standard.ai/scalable-machine-learning-with-apache-spark/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/scalable-machine-learning-with-apache-spark/v02

Setup completed (6 seconds)





## Load the Dataset
For this example, we will use a new COVID-19 dataset. Run the cell below to create our dataframe **`covid_df`**.

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

file_path = f"dbfs:/mnt/dbacademy-datasets/scalable-machine-learning-with-apache-spark/v02/COVID/coronavirusdataset/Time.csv"
covid_df = (spark.read
            .format("csv")
            .option("header",True)
            .option("inferSchema", True)
            .load(file_path)
            .withColumn("index", monotonically_increasing_id()))

display(covid_df)

date,time,test,negative,confirmed,released,deceased,index
2020-01-20,16,1,0,1,0,0,0
2020-01-21,16,1,0,1,0,0,1
2020-01-22,16,4,3,1,0,0,2
2020-01-23,16,22,21,1,0,0,3
2020-01-24,16,27,25,2,0,0,4
2020-01-25,16,27,25,2,0,0,5
2020-01-26,16,51,47,3,0,0,6
2020-01-27,16,61,56,4,0,0,7
2020-01-28,16,116,97,4,0,0,8
2020-01-29,16,187,155,4,0,0,9





Run the cell below to set up a database and unique table name **`table_name`** for the lab.

In [0]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS charlie_ohara_4mi2_da_sml")
table_name = f"charlie_ohara_4mi2_da_sml.coronavirus"

print(f"Table Name: {table_name}")

Table Name: charlie_ohara_4mi2_da_sml.coronavirus





Let's set up our FeatureStoreClient **`fs`**. 

To create a feature store client, initialize a **`FeatureStoreClient`** object from the **`feature_store`** module.

In [0]:
# TODO
from databricks import feature_store
 
fs = feature_store.FeatureStoreClient()




## Extract Features

In this simple example we want to predict the number of daily deceased using the other information from from day. 

Before we write to our feature table, we will need to write a feature computation function that separates our features from the label. 

Fill in the feature computation function below to select only the feature columns, not **`deceased`**.

In [0]:
columns = covid_df.columns
print(type(columns))
columns.remove('deceased')


<class 'list'>


In [0]:
@feature_store.feature_table
def select_features(dataframe):
  return dataframe.select(columns)


covid_features_df = select_features(covid_df)
display(covid_features_df)

date,time,test,negative,confirmed,released,index
2020-01-20,16,1,0,1,0,0
2020-01-21,16,1,0,1,0,1
2020-01-22,16,4,3,1,0,2
2020-01-23,16,22,21,1,0,3
2020-01-24,16,27,25,2,0,4
2020-01-25,16,27,25,2,0,5
2020-01-26,16,51,47,3,0,6
2020-01-27,16,61,56,4,0,7
2020-01-28,16,116,97,4,0,8
2020-01-29,16,187,155,4,0,9





## Create Feature Table

Now that we have our features ready, complete the cell below to create our feature table.

Make sure to set the name to the **`table_name`** we defined above.

**NOTE:** The primary key needs to be defined in a list as follows: ["primary key name"]

In [0]:

fs.create_table(
    name=table_name,
    primary_keys=["index"],
    df=covid_features_df,
    schema=covid_features_df.schema,
    description="lab"
)

2024/03/01 16:45:38 INFO databricks.feature_store._compute_client._compute_client: Created feature table 'hive_metastore.charlie_ohara_4mi2_da_sml.coronavirus'.
/databricks/python/lib/python3.9/site-packages/databricks/feature_store/entities/_feature_store_object.py:8: FutureWarning: ``databricks.feature_store.entities.feature_table.FeatureTable.keys`` is deprecated since v0.3.6. This method will be removed in a future release. Use ``FeatureTable.primary_keys`` instead.
  yield prop, self.__getattribute__(prop)


Out[11]: <FeatureTable: keys=['index'], tags={}>




## Update Feature Table

Imagine now that we wanted to add separate columns for the month and day of the date for each entry. 

Rather than recompute the table with these values, we just want to append these new columns to the existing table. 

First, let's create columns for the month and day.

In [0]:
from pyspark.sql.functions import month, dayofmonth

add_df = (covid_features_df
  .select("date", "index")
  .withColumn("month", month("date"))
  .withColumn("day", dayofmonth("date"))
)

display(add_df)

date,index,month,day
2020-01-20,0,1,20
2020-01-21,1,1,21
2020-01-22,2,1,22
2020-01-23,3,1,23
2020-01-24,4,1,24
2020-01-25,5,1,25
2020-01-26,6,1,26
2020-01-27,7,1,27
2020-01-28,8,1,28
2020-01-29,9,1,29





Now we want to add this information to our feature table using **`write_table`**. 

**NOTE:** Remember, we can use either **`"overwrite"`** or **`"merge"`** mode. Which one should we use here?

In [0]:
# added 2 columns
fs.write_table(
    name=table_name,
    df=add_df,
)




Now try using **`fs.read_table`**, specifying the **`table_name`** to see our updated feature table.

In [0]:
# TODO
updated_df = fs.read_table(table_name)

display(updated_df)

date,time,test,negative,confirmed,released,index,month,day
2020-01-20,16,1,0,1,0,0,1,20
2020-01-21,16,1,0,1,0,1,1,21
2020-01-22,16,4,3,1,0,2,1,22
2020-01-23,16,22,21,1,0,3,1,23
2020-01-24,16,27,25,2,0,4,1,24
2020-01-25,16,27,25,2,0,5,1,25
2020-01-26,16,51,47,3,0,6,1,26
2020-01-27,16,61,56,4,0,7,1,27
2020-01-28,16,116,97,4,0,8,1,28
2020-01-29,16,187,155,4,0,9,1,29





## Training 

Now that we have our feature table, we are ready to use it for model training. We'll need our target variable **`deceased`** in addition to our features, so let's get that first.

In [0]:
target_df = covid_df.select(["index", "deceased"])

display(target_df)

index,deceased
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0





Now let's create our training and test datasets.

In [0]:
from sklearn.model_selection import train_test_split

def load_data(table_name, lookup_key):
    model_feature_lookups = [feature_store.FeatureLookup(table_name=table_name, lookup_key=lookup_key)]

    # fs.create_training_set will look up features in model_feature_lookups with matched key from inference_data_df
    training_set = fs.create_training_set(target_df, model_feature_lookups, label="deceased", exclude_columns=["index","date"])
    training_pd = training_set.load_df().toPandas()

    # Create train and test datasets
    X = training_pd.drop("deceased", axis=1)
    y = training_pd["deceased"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, training_set

X_train, X_test, y_train, y_test, training_set = load_data(table_name, "index")
X_train.head()

,confirmed,day,month,negative,released,test,time
84,10537,13,4,494815,7447,518743,0
2,1,22,1,3,0,4,16
94,10702,23,4,563130,8411,583971,0
45,5766,5,3,118965,88,146541,0
42,4212,2,3,71580,31,109591,0





Now we can train a model and register it to the feature store.

In [0]:
from mlflow.tracking.client import MlflowClient

client = MlflowClient()

suffix = DA.unique_name("-")
model_name = f"feature-store-covid_{suffix}"

try:
    client.delete_registered_model(model_name) # Deleting model if already created
except:
    None

In [0]:
import mlflow
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from mlflow.models.signature import infer_signature

def train_model(table_name):
    # load data 
    X_train, X_test, y_train, y_test, training_set = load_data(table_name, "index")

    ## fit and log model
    with mlflow.start_run() as run:
        # define algo to use
        rf = RandomForestRegressor(max_depth=3, n_estimators=20, random_state=42)
        # create the model
        rf.fit(X_train, y_train)
        # use the model to predict 
        y_pred = rf.predict(X_test)
        # evaluate performance
        mlflow.log_metric("mse", mean_squared_error(y_test, y_pred))
        mlflow.log_metric("r2", r2_score(y_test, y_pred))

        # add to feature store so we can see which features were used to create which model
        fs.log_model(
            model=rf,
            artifact_path="feature-store-model",
            flavor=mlflow.sklearn,
            training_set=training_set,
            registered_model_name=model_name,
            input_example=X_train[:5],
            signature=infer_signature(X_train, y_train)
        )
    
train_model(table_name)

/databricks/python/lib/python3.9/site-packages/mlflow/models/signature.py:130: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)
/databricks/python/lib/python3.9/site-packages/mlflow/models/signature.py:131: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input 




Now we have a trained model! Check the Feature Store UI to see that our model is now there. Can you tell which features this model uses from that table and which we excluded?

Finally, let's apply the model.

In [0]:
## For sake of simplicity, we will just predict on the same inference_data_df
batch_input_df = target_df.drop("deceased") # Exclude true label
# make predictions based on the features using the registered model
predictions_df = fs.score_batch(f"models:/{model_name}/1", 
                                  batch_input_df, result_type="double")
display(predictions_df)

2024/03/01 16:52:06 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.
2024/03/01 16:52:06 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


index,confirmed,day,month,negative,released,test,time,prediction
0,1,20,1,0,0,1,16,1.947045450366538
1,1,21,1,0,0,1,16,1.947045450366538
2,1,22,1,3,0,4,16,1.947045450366538
3,1,23,1,21,0,22,16,1.947045450366538
4,2,24,1,25,0,27,16,1.947045450366538
5,2,25,1,25,0,27,16,1.947045450366538
6,3,26,1,47,0,51,16,1.947045450366538
7,4,27,1,56,0,61,16,1.947045450366538
8,4,28,1,97,0,116,16,1.947045450366538
9,4,29,1,155,0,187,16,1.947045450366538


## Classroom Cleanup

Run the following cell to remove lessons-specific assets created during this lesson:

In [0]:
DA.cleanup()

&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>